# Building a survival model for the Titanic Kaggle competition

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

ModuleNotFoundError: No module named 'seaborn'

### Importing the data

In [ ]:
# Read data out into train and test dataframes
train = pd.read_csv('/kaggle/input/titanic/train.csv', index_col='PassengerId')
test = pd.read_csv('/kaggle/input/titanic/test.csv', index_col='PassengerId')

### Exploring the data

In [ ]:
# Print out the top 5 rows of the training data
train.head()

In [ ]:
# Print out the top 5 rows of the test data
test.head()

In [ ]:
# look at the datatypes and null values for the training data
train.info()

In [ ]:
# Look at the columns present in both training and test data
print('Train data columns: ', train.columns)
print('Test data columns: ', test.columns)

In [ ]:
# Look at what data is categorical data
s = (train.dtypes == 'object')
object_cols = list(s[s].index)
print('Categorical columns: ', object_cols)

### Exploring relationships in the data

In [ ]:
sns.lmplot(x='Age', y='Fare', hue='Survived', data=train)
plt.title('Realtionship between age and fare split by survival on the Titanic')

We can a positive coloration for those who surrived between thier age and the fare they paid. Showing that older passengers tended to pay more for their fare. We  However for those who did not surrive while still positive this relationship is less evident.

In [ ]:
sns.lmplot(x='Parch', y='SibSp', hue='Survived', data=train)
plt.title('Realtionship between having siblings or spouse with you and being a parent or child split by survival on the Titanic')

In both those who surrived and those who did not there is a positive correlation between a passenger having more siblings or spouse and being a parent or child, perhaps indicating the amount of families travelling aboard.

In [ ]:
sns.lmplot(x='Age', y='SibSp', hue='Survived', data=train)
plt.title('Realtionship between age and having a sibling or spouse split by survival on the Titanic')

Here regardless of your surrival the older you were the less likely you had a sibling or spouse with you.

In [ ]:
sex_grouped_by_survival = train.groupby(['Survived']).Sex.value_counts().sort_index()
sex_grouped_by_survival

In [ ]:
survived = plt.bar(range(len(sex_grouped_by_survival[0])), sex_grouped_by_survival[0], color='purple')
died = plt.bar(range(len(sex_grouped_by_survival[1])), sex_grouped_by_survival[1], color='yellow', bottom=sex_grouped_by_survival[0])
plt.legend([survived, died], ['Died', 'Survived'])
labels = ['Female', 'Male', ]
plt.xticks(range(len(sex_grouped_by_survival[1])), labels)
plt.ylabel('Number of passengers')
plt.title('Realtionship between a passengers Sex and survival on the Titanic')
plt.show()

### Looking at the relationship class and sex has on survival on the Titanic

In [ ]:
print(train.groupby(['Sex', 'Pclass']).mean()['Survived'])
print(train.groupby(['Sex', 'Pclass']).std()['Survived'])

In [ ]:
train['Survival'] = train.Survived.map(lambda t: 'Survived' if t == 1 else 'Died')
train['Class'] = train.Pclass.map(lambda t: '1st class' if t == 1 else ('2nd class' if t == 2 else '3rd class'))

(train
.groupby(['Survival', 'Class', 'Sex'])
.Class
.count()
.plot(kind='bar')
)

The largest group from the Titanic by far was 3rd class males who died by quite a gap. We can also see that females in 1st and 2nd class more often survived than died but females from 3rd class appear to be split in half between those which survived and those who did not.

### Female Passengers statistics (for the training data)

In [ ]:
#Number of Female passengers
female_passengers = train.loc[train['Sex'] == 'female']
print('Number of female passengers in training data: ', len(female_passengers))

# Number of female passengers under 18
female_passengers_under_18 = female_passengers.loc[female_passengers['Age'] < 18]
print('Number of female passengers under 18 in training data: ', len(female_passengers_under_18))

# Number of female passengers over 60
female_passengers_over_60 = female_passengers.loc[female_passengers['Age'] > 60]
print('Number of female passengers over 60 in training data: ', len(female_passengers_over_60))

# Number of female passengers traveling with a parent or child
female_passengers_parch = female_passengers.loc[female_passengers['Parch'] >= 1]
print('Number of female passengers traveling with a parent or child in training data: ', len(female_passengers_parch))

# Number of female passengers traveling with a sibling or spouse
female_passengers_with_sibsp = female_passengers.loc[female_passengers['SibSp'] >= 1]
print('Number of female passengers traveling with a sibling or spouse in training data: ', len(female_passengers_with_sibsp))

# Number of female passengers who survived
female_passengers_that_survived = female_passengers.loc[female_passengers['Survived'] == 1]
print('Number of female passengers that survived in training data: ', len(female_passengers_that_survived))

# Percentage of female passengers that survived f'{x:.0%}'
print('Precentage of female passengers that survived in training data: ', f'{len(female_passengers_that_survived)/len(female_passengers)*100:.1f}%')

The difference between class and surrival is not as large as I would have expected. I wonder if this graph is somewhat decieving as it's hard to read what was the truth for those in 3rd class. Did none surrive? It's hard to say. Only that the majority of surrivers were in the first and second class.

In [ ]:
# Extract surname into separate column
train['Surname'] = train['Name'].map(lambda x: x.split(',')[0])
print(train['Surname'][0:5])

In [ ]:
# Extract title into separate column
train['Title'] = train['Name'].map(lambda x: x.split(',')[1].split('.')[0].strip())
train['Title'] = train['Title'].map(lambda x: 'Countess' if x == 'the Countess' else x)
# Mlle == Mademoiselle
# Don ==  A head, tutor, or fellow at a college of Oxford or Cambridge. Or title for males in spanish speaking areas
for title in train['Title'].unique():
    print(title, train.loc[(train['Title'] == title)].size)

In [ ]:
plt.title('Title distribution of those who survived on the Titanic')
(train[train.Survived == 1]
.groupby(['Title'])
.Title
.count()
.plot(kind='bar')
)

In [ ]:
plt.title('Title distribution of those who died on the Titanic')
(train[train.Survived == 0]
.groupby(['Title'])
.Title
.count()
.plot(kind='bar')
)

It's hard to say if there is a correlation between title and survival. We can see see that the largest group of those who died held the title Mr. This relationship could be indirectly, for example master is a title for young men and more children surrived than adults.

In [ ]:
# Extract first name into separate column
train['First_names'] = train['Name'].map(lambda x: x.split('.')[1].split('(')[0].strip())
print(train['First_names'][0:5])

In [ ]:
number_of_passengers_named_john = len(train.loc[train['First_names'] == 'John'])

In [ ]:
(train
.groupby(['Survival', 'First_names'])
.First_names
.count()
.sort_values(ascending=False)
[:10]
)

While it remains unlikely someone's first name would play a role in their survival on the Titanic interesting all of the passengers in the training data called John died. This isn't statistically so suprising as they were most likely male passengers and we've already discovered male passengers were predominently in the group which died. It may point to patterns in the names given to different classes if indeed such a thing existed.

In [ ]:
# Extract alternative name into separate column
train['Alternative_name'] = train['Name'].map(lambda x: x.split('(')[1].split(')')[0] if '(' in x else '')
print(train['Alternative_name'][0:5])

In [ ]:
# Extract alternative surname (most likely maiden name) into separate column
train['Alternative_surname'] = train['Alternative_name'].map(lambda alt_name: '' if alt_name == '' else alt_name.split()[-1])
for alt_surname in train['Alternative_surname'].unique():
    print(alt_surname, train.loc[(train['Alternative_surname'] == alt_surname)].Alternative_surname.count())

In [ ]:
# Look at how many values there are for cabin
for cabin in train['Cabin'].unique():
    print(cabin, train.loc[(train['Cabin'] == cabin)].size)

In [ ]:
# Extract which deck a passager had their cabin based on the Cabin number
train['Deck']= train['Cabin'].map(lambda x: 'Unknown' if str(x) == 'nan' else str(x)[0])

In [ ]:
plt.title('Realtionship between Deck and survival on the Titanic')
(train[train.Deck != 'Unknown']
.groupby(['Survival', 'Deck'])
.Deck
.count()
.plot(kind='bar')
)

This maybe gives some insight to how decks were labelled on the titanic. We would ideally want to look at a map to see where the decks were to read more into these results. Did decks divide classes, did they also divide workers from passengers? Did all decks have the same amount of occupants? However it seems surrival seemed more likely if you were on deck E, D, B.

In [ ]:
deck_data = train[train.Deck != 'Unknown'].groupby(['Deck', 'Survival']).Deck.count() # remove unknowns as they skew the data
deck_data


In [ ]:
deck_survival_ratio = pd.Series()

for deck in train.Deck:
    survivors = train[(train.Deck == deck) & (train.Survived == 1)].Deck.count()
    total_deck_passengers = train[train.Deck == deck].Deck.count()
    ratio = (survivors/total_deck_passengers)*100
    deck_survival_ratio[deck] = ratio

plt.title('Ratio of survival per deck on the Titanic')
deck_survival_ratio.sort_index().plot(kind='bar')

In [ ]:
# Extract the ticket type (assuming potentially where they bought the ticket) from the ticket value
train['Ticket_type'] = train['Ticket'].map(lambda x: x.split()[0] if any(c.isalpha() for c in x.split()[0]) else 'Unknown')
print(train['Ticket_type'].unique())
print(train['Ticket_type'].unique().size)

In [ ]:
plt.figure(figsize=(16,10))
plt.title('Realtionship between Ticket type and survival on the Titanic')
(train[train.Ticket_type != 'Unknown']
.groupby(['Survival', 'Ticket_type'])
.Ticket_type
.count()
.plot(kind='bar')
)

In [ ]:
# Look at how many values there are for ticket tzpe
for ticket in train['Ticket_type'].unique():
    print(ticket, train.loc[(train['Ticket_type'] == ticket)].Ticket_type.count())

In [ ]:
# Put feature columns into X_train dataframe
# base_features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Surname', 'Title', 'Alternative_surname', 'Deck', 'Ticket_type']
base_features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Title', 'Deck', 'Ticket_type']
X_train = train[base_features].copy()
X_train.index = train.index
# Put survival value into y
y = train['Survived']

print(y.head())
X_train.head()

In [ ]:
# Look at what data is categorical data
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)
print(object_cols)

In [ ]:
# Update the Embarked column to replace nan values with unknown
X_train['Embarked'] = X_train['Embarked'].fillna('unknown')

print(X_train['Embarked'].unique())

In [ ]:
# Update the Age column to replace nan values with the mean value
X_train['Age'].fillna(X_train['Age'].mean(), inplace=True)

In [ ]:
# Check cardinality of categorical variables
object_nunique = list(map(lambda col: X_train[col].nunique(), object_cols))
d = dict(zip(object_cols, object_nunique))
# Print in ascending order
sorted(d.items(), key=lambda x: x[1])

# We can conclude that for those with low cardinality we can use one-hot encoding and for the time being I am dropping those with high cardinality

In [ ]:
# Assign categorical data columns to low or high cardinality lists
low_cardinality = ['Deck', 'Title', 'Sex', 'Embarked', 'Ticket_type']
# high_cardinality  = ['Ticket_type', 'Alternative_surname', 'Surname']

In [ ]:
# from sklearn.preprocessing import LabelEncoder
# label_encoder = LabelEncoder()
# for col in high_cardinality:
#     X_train[col] = label_encoder.fit_transform(X_train[col])

X_train.head()
X_train.info()

In [ ]:
from sklearn.preprocessing import OneHotEncoder
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[low_cardinality]))
print('OH_cols_train: ', OH_cols_train.columns.tolist(), ' shape ', OH_cols_train.shape)
non_oh_encoded = X_train.drop(low_cardinality, axis=1)
OH_cols_train.index = X_train.index
print('non_oh_encoded: ', non_oh_encoded.columns.tolist(), ' shape ', non_oh_encoded.shape)
OH_X_train = pd.concat([non_oh_encoded, OH_cols_train], axis=1)
OH_X_train.index = X_train.index
print('OH_X_train: ', OH_X_train.columns.tolist(), ' shape ', OH_X_train.shape)
OH_X_train.columns

In [ ]:
# update column names to string type
OH_X_train.columns = OH_X_train.columns.astype(str)

In [ ]:
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(OH_X_train, y, random_state = 0)

In [ ]:
val_X.dropna(axis=0, how='any', inplace=True)

In [ ]:
print(train_X.columns)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(random_state=0)
rf_model.fit(train_X, train_y.values.ravel())

In [ ]:
predictions = rf_model.predict(val_X)

In [ ]:
print(predictions[0:5])
print(val_y[0:5])
print(f'Accuracy: {rf_model.score(val_X, val_y):.2f}%')

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(rf_model, random_state=1).fit(val_X, val_y)

eli5.show_weights(perm, feature_names=val_X.columns.tolist())

### Permutation importance of features

Here it appears that Pclass, Fare, Age and sibsp have relatively high importance as features. One or two of the one-hot-encoded values also appear to have a high importance. With the current one-hot-encoder I unfortunately cannot tell which one these refers to.

In [ ]:
important_features = ['Pclass', 'Fare', 'Age','SibSp']

In [ ]:
from sklearn.model_selection import train_test_split
train_reduced_features_X, val_reduced_features_X, train_reduced_features_y, val_reduced_features_y = train_test_split(OH_X_train[important_features], y, random_state = 0)
val_X.dropna(axis=0, how='any', inplace=True)

from sklearn.ensemble import RandomForestClassifier
rf_reduced_features_model = RandomForestClassifier(random_state=0)
rf_reduced_features_model.fit(train_reduced_features_X, train_reduced_features_y.values.ravel())

predictions = rf_reduced_features_model.predict(val_reduced_features_X)

print('Predictions: ', predictions[0:5])
print('Y Values: ', val_reduced_features_y[0:5])
print(f'Accuracy: {rf_reduced_features_model.score(val_reduced_features_X, val_reduced_features_y):.2f}%')

We can see here our accuracy has actually been reduced by using only the features that the permutation said were important.

### Partial dependence plots

In [ ]:
from pdpbox import pdp, get_dataset, info_plots


for feature_name in important_features:
    pdp_goals = pdp.pdp_isolate(model=rf_model, dataset=val_X, model_features=val_X.columns.tolist(), feature=feature_name)
    pdp.pdp_plot(pdp_goals, feature_name)
    plt.show()

### 2D Partial Dependance Plots

In [ ]:
inter1 = pdp.pdp_interact(model=rf_model, dataset=val_X, model_features=val_X.columns.tolist(), features=important_features)
pdp.pdp_interact_plot(pdp_interact_out=inter1, feature_names=val_X.columns.tolist(), plot_type='contour')
plt.show()

### SHAP Values
break down a prediction to show the impact of each feature.

In [ ]:
row_to_show = 5
data_for_prediction = val_X.iloc[row_to_show]  # use 1 row of data here. Could use multiple rows if desired
data_for_prediction_array = data_for_prediction.values.reshape(1, -1)


rf_model.predict_proba(data_for_prediction_array)

In [ ]:
import shap
explainer = shap.TreeExplainer(rf_model)
shap_values = explainer.shap_values(data_for_prediction)
shap.initjs()
data_for_prediction = val_X.iloc[1]
shap.force_plot(explainer.expected_value[1], shap_values[1], data_for_prediction) # looking at values for those that surrived
#  Shap values show how much a given feature changed our prediction (compared to if we made that prediction at some baseline value of that feature).

In [ ]:
# Create object that can calculate shap values
explainer = shap.TreeExplainer(rf_model)

# calculate shap values. This is what we will plot.
# Calculate shap_values for all of val_X rather than a single row, to have more data for plot.
shap_values = explainer.shap_values(val_X)

# Make plot. Index of [1] is explained in text below.
shap.summary_plot(shap_values[1], val_X)

In [ ]:
# # Create object that can calculate shap values
# explainer = shap.TreeExplainer(my_model)

# # calculate shap values. This is what we will plot.
# shap_values = explainer.shap_values(X)

# make plot.
shap.dependence_plot('Age', shap_values[1], val_X, interaction_index="Pclass")